In [4]:
# pip install llama-index-llms-huggingface-api llama-index-embeddings-huggingface

In [5]:
# !pip install python-dotenv

In [6]:
# !pip install llama-index-llms-groq

In [9]:
# pip install llama-index-vector-stores-chroma

In [14]:
# pip install llama-index-tools-google

In [21]:
!pip install llama-index-tools-model-context

ERROR: Could not find a version that satisfies the requirement llama-index-tools-model-context (from versions: none)
ERROR: No matching distribution found for llama-index-tools-model-context


## Simple setup

In [11]:
from llama_index.llms.groq import Groq
import os
from google.colab import userdata

llm = Groq(model="llama3-70b-8192", api_key=userdata.get('groq_api'))

response = llm.complete("Hello, how are you?")
print(response)


I'm just a language model, I don't have emotions or feelings like humans do, so I don't have good or bad days. However, I'm functioning properly and ready to assist you with any questions or tasks you may have! How can I help you today?


## Creating a FunctionTool


In [7]:
from llama_index.core.tools import FunctionTool

def get_weather(location: str) -> str:
    """Useful for getting the weather for a given location."""
    print(f"Getting weather for {location}")
    return f"The weather in {location} is sunny"

tool = FunctionTool.from_defaults(
    get_weather,
    name="my_weather_tool",
    description="Useful for getting the weather for a given location.",
)
tool.call("New York")

Getting weather for New York


ToolOutput(content='The weather in New York is sunny', tool_name='my_weather_tool', raw_input={'args': ('New York',), 'kwargs': {}}, raw_output='The weather in New York is sunny', is_error=False)

## Creating a QueryEngineTool


In [12]:
from llama_index.core import VectorStoreIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

embed_model = HuggingFaceEmbedding("BAAI/bge-small-en-v1.5")

db = chromadb.PersistentClient(path="./alfred_chroma_db")
chroma_collection = db.get_or_create_collection("alfred")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

index = VectorStoreIndex.from_vector_store(vector_store, embed_model=embed_model)

query_engine = index.as_query_engine(llm=llm)
tool = QueryEngineTool.from_defaults(query_engine, name="some useful name", description="some useful description")

## Creating Toolspecs

In [15]:
from llama_index.tools.google import GmailToolSpec

tool_spec = GmailToolSpec()
tool_spec_list = tool_spec.to_tool_list()

To get a more detailed view of the tools, we can take a look at the metadata of each tool.



In [16]:
[(tool.metadata.name, tool.metadata.description) for tool in tool_spec_list]

[('load_data',
  "load_data() -> List[llama_index.core.schema.Document]\nLoad emails from the user's account."),
 ('search_messages',
  "search_messages(query: str, max_results: Optional[int] = None)\nSearches email messages given a query string and the maximum number\n        of results requested by the user\n           Returns: List of relevant message objects up to the maximum number of results.\n\n        Args:\n            query[str]: The user's query\n            max_results (Optional[int]): The maximum number of search results\n            to return.\n        "),
 ('create_draft',
  "create_draft(to: Optional[List[str]] = None, subject: Optional[str] = None, message: Optional[str] = None) -> str\nCreate and insert a draft email.\n           Print the returned draft's message and id.\n           Returns: Draft object, including draft id and message meta data.\n\n        Args:\n            to (Optional[str]): The email addresses to send the message to\n            subject (Optiona

In [19]:
from llama_index.tools.google import GoogleSearchToolSpec

google_api_key = "YOUR_GOOGLE_API_KEY"
search_engine_id = "YOUR_SEARCH_ENGINE_ID"

tool_spec = GoogleSearchToolSpec(key=google_api_key, engine=search_engine_id)
tool_spec_list = tool_spec.to_tool_list()

## To get a more detailed view of the tools, we can take a look at the metadata of each tool.

[(tool.metadata.name, tool.metadata.description) for tool in tool_spec_list]

## Model Context Protocol (MCP) in LlamaIndex

In [22]:
# from llama_index.tools.mcp import BasicMCPClient, McpToolSpec

# # We consider there is a mcp server running on 127.0.0.1:8000, or you can use the mcp client to connect to your own mcp server.
# mcp_client = BasicMCPClient("http://127.0.0.1:8000/sse")
# mcp_tool = McpToolSpec(client=mcp_client)

# # get the agent
# agent = await get_agent(mcp_tool)

# # create the agent context
# agent_context = Context(agent)